In [1]:
import cv2
import numpy as np

In [ ]:
# preparation
confidents = []
bounding_box = []
classes = []
probability = 0.5
thredhold = 0.4

## Load class label 
with open("obj.names") as f:
    labels = [line.strip() for line in f]
    
cap = cv2.VideoCapture("sample1.mp4")

while True:
    ret,frame = cap.read()
    ## img shape
    h,w,_ = frame.shape
    blob = cv2.dnn.blobFromImage(frame,1./255,(416,416),swapRB=True,crop=False)
    net = cv2.dnn.readNetFromDarknet("yolov4-obj.cfg","yolov4-obj_last.weights")
#     net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#     net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    # layers_names = net.getLayerNames()
    output_layers = net.getUnconnectedOutLayersNames()
    input_img = net.setInput(blob)

    ## Feed input to network
    output = net.forward(output_layers)

    ## Extract result from each output layer
    for result in output:
        for obj in result:
            scores = obj[5:]
            class_current = np.argmax(scores)
            class_confident_score = scores[class_current]
            if class_confident_score > probability:
                box_center_x,box_center_y,box_width,box_height = (obj[0]*w),(obj[1]*h),(obj[2]*w),(obj[3]*h)
                box_origin_x,box_origin_y = box_center_x-(0.5*box_width),box_center_y-(0.5*box_height)
                bounding_box.append([int(box_origin_x),int(box_origin_y),int(box_width),int(box_height)])
                confidents.append(float(class_confident_score))
                classes.append(class_current)
                
    ## Non_maximum_supression
    results = cv2.dnn.NMSBoxes(bounding_box,confidents,probability,thredhold)
    color = np.random.uniform(0,255,size=(len(labels),3))
    if len(results) > 0:
        for i in results.flatten():
            text = f'{labels[classes[i]]}: {confidents[i]}'
            cv2.rectangle(frame,(bounding_box[i][0],bounding_box[i][1]),
                          (bounding_box[i][0]+bounding_box[i][2],bounding_box[i][1]+bounding_box[i][3]),(0,0,125),thickness=2)
            cv2.putText(frame,text,(bounding_box[i][0],bounding_box[i][1]),cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,125), 2)

        cv2.imshow("detection",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
              break
cap.release()
cv2.destroyAllWindows()   

In [ ]:
cap = cv2.VideoCapture("sample1.mp4")

while True:
    ret,frame = cap.read()
    ## img shape
    cv2.imshow("detection",frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()   

In [ ]:
count = cv2.cuda.getCudaEnabledDeviceCount()
print(count)